# Enumerating Zonotope Vertices With a Randomized Algorithm

__Kerrick Stinson__, Colorado School of Mines, <kstinson@mines.edu>

__David Gleich__, Purdue University, <dglecih@purdue.edu>

__Paul Constantine__, Colorado School of Mines, <pconstan@mines.edu>

__Ryan Howard__, Colorado School of Mines, <ryhoward@mines.edu>

<br>

In this notebook, we'll be summarizing a randomized algorithm for enumerating zonotope vertices, presented in [[1]][R1].

### Introduction

Zonotopes are low-dimensional linear projections of hypercubes that have various applications. These applications often require enumeration of the vertices of the low-dimensional zonotope, which are a subset of the projected vertices of the high-dimensional hypercube. However, current algorithms' complexities tend to increase rapidly as the hypercube's dimension increases, which can make vertex enumeration computationally infeasible. Here, we describe a randomized algorithm that can be used for vertex enumeration and that has several favorable properties, such as approximation of the zonotope upon premature termination and a tendency to find the most 'important' vertices before the less important ones.

### Zonotopes

Let $\mathbf A\in\mathcal R^{n\times m}$, with $n<m$, and define the zonotope $Z\subset \mathcal R^n$ as $Z = Z(\mathbf A) = \{\mathbf{Ax}\ |\ \mathbf x\in[-1,1]^m\}$. $[-1,1]^m$ is an $m$-dimensional cube and $\{\mathbf{Ax}\ |\ \mathbf x\in[-1,1]^m\}$ represents the (linear) projection of this cube onto $\mathcal R^n$ according to the matrix $\mathbf A$ (called the _generator matrix_). We can equivalently define the zonotope as the Minkowski sum of $m$ line segments in $\mathcal R^n$: $Z = A_1 + \cdots + A_m$, where $A_i = \{\gamma\mathbf a_i\ |\ \gamma\in[-1,1]\}$ and $\mathbf a_i$ is the $i^{th}$ column of $\mathbf A$.

Zonotopes have application in many fields, such as collision detection, control systems, compressed sensing, integer quadratic programming, signal processing, and regression models. A particular novel application of interest arises from approximating a multivariate function in fewer dimensions; i.e. if $f(\mathbf x)$ is a function from $\mathcal R^m\rightarrow \mathcal R$, we can approximate $f$ by $f(\mathbf x)\approx g(\mathbf{Ax})$ where $\mathbf A\in\mathcal R^{n\times m}$ and $g$ is a function from $\mathcal R^n\rightarrow \mathcal R$. In the methodology of _active subspaces_ ([[2]][R2]), $\mathbf A$ is a matrix identifying the most 'important' directions in the input space of $f$ and if $\mathbf x\in[-1,1]^m$, then the domain of $g$ is a zonotope in $\mathcal R^n$.

Several algorithms for vertex enumeration or zonotpe approximation exist, such as a general convex hull algorithm or a reverse search algorithm, whose complexities tend to increase rapidly with $m$, or Goffin's algorithm for approximating zonotopes with ellipses, which is difficult to implement numerically. We propose the following randomized algorithm for enumerating a zonotope's vertices ($\text{sign}(\mathbf v)$ returns a vector of elements in $\{-1,1\}$ corresponding to the signs of $\mathbf v$'s components):

1. Let $k$ be the number of vertices of the zonotope and initialize the set of vertices $V=\emptyset$.
2. While $|V| < k$ do:
    1. Draw $\mathbf x$ independently from a standard normal distribution on $\mathcal R^n$.
    2. Compute $\mathbf v_+\equiv \mathbf A\text{sign}(\mathbf A^T\mathbf x)$ and $\mathbf v_-\equiv -\mathbf v_+$.
    3. If $\mathbf v_+\not\in V$, add $\mathbf v_+$ and $\mathbf v_-$ to $V$.
    
If this algorithm is terminated before $|V|=k$, the convex hull of the elements of $V$ are contained within the target zonotope.

### Zonotope Vertices and our Algorithm

For the following discussion, let $\mathbf A$ generate the zonotope $Z$ and assume $\mathbf A$ has no column of all zeros and no two columns are scalar multiples of each other. Then the following results hold: first, for any $\mathbf x\in\mathcal R^n$ such that $\mathbf{Ax}$ has no zero components, $\mathbf v = \mathbf m(\mathbf x)\equiv\mathbf A\text{sign}(\mathbf{Ax})$ is a vertex of $Z$; second, if $\mathbf v$ is a vertex of $Z$, then so is $-\mathbf v$; and third, defining $H$ as the set of $\mathbf x\in\mathcal R^n$ orthogonal to any of $\mathbf A$'s columns, the mapping $\mathbf m:\mathcal R^n\backslash H\rightarrow \text{vert}(Z(\mathbf A))$ is well-defined and onto ($\text{vert}(Z)$ is the set of $Z$'s vertices).

We now examine the conditions for which our algorithm returns a particular vertex. First, recall that the normal cone of a vertex, $\mathbf v$, is defined as $N_Z(\mathbf v) = \{\mathbf x\in\mathcal R^n\ |\ <\mathbf x, \mathbf z - \mathbf v>\leq 0\ \forall\mathbf z\in Z\}$. The region for which an $\mathbf x$ will return a particular vertex, $\mathbf v$, is the interior of the vertex's normal cone: $\mathbf m^{-1}(\mathbf v)=\text{int}(N_Z(\mathbf v))$. Geometrically, a vertex's normal cone is larger if the zonotope is more pointed at that vertex and smaller if the zonotope is relatively flat at that vertex. This means our algorithm will more likely identify pointy vertices (which are more important in characterizing the zonotope's shape) than flat vertices (which don't affect overall shape as much).

### References:

[[1]][R1] K. Stinson, D.F. Gleich, and P.G. Constantine. _A randomized algorithm for  numerating zonotope vertices_. arXiv:1602.06620, 2016

[[2]][R2] P.G. Constantine, E. Dow, and Q. Wang. _Active subspaces in theory and practice: applications to kriging surfaces_. SIAM J. Sci. Comput., 36(4), A1500–A1524, 2014

[R1]: https://arxiv.org/abs/1602.06620
[R2]: http://dx.doi.org/10.1137/130916138

### Acknowledgments

This material is based upon work supported by the U.S. Department of Energy Office of Science, Office of Advanced Scientific Computing Research, Applied Mathematics program under Award Number DE-SC-0011077.

<br>

We'll now present some numerical experiments comparing our algorithm to a reverse search algorithm implemented in Christophe Weibel's [MinkSum](https://sites.google.com/site/christopheweibel/research/minksum) library. We use [Benjamin Kern's](http://ifatwww.et.uni-magdeburg.de/syst/about_us/people/kern/) Python wrapper for MinkSum, PyMINKSUM, which you will need to install if you wish to run this notebook.

### Installing PyMINKSUM

To install PyMINKSUM, follow the following steps (tested on Ubuntu 14.04 LTS 64-bit; other operating systems may require alterations to this process):
1. Download gmp-4.3.2 ([.tar.gz](https://ftp.gnu.org/gnu/gmp/gmp-4.3.2.tar.gz) or [.tar.bz2](https://ftp.gnu.org/gnu/gmp/gmp-4.3.2.tar.bz2)) and extract to a directory of your choosing.
2. Navigate into the newly created directory ('gmp-4.3.2/') in the terminal and execute the commands `./configure` and `sudo make install`.
3. Download PyMINKSUM [here](http://ifatwww.et.uni-magdeburg.de/syst/about_us/people/kern/downloads/pyminksum-0.11.zip) and extract to a directory of your choosing.
4. In the terminal, navigate to the folder 'pyminksum-0.11/external-src/MINKSUM_1.6.2/external/' and execute the the command `make` (some errors or warnings might be present here; installation should still work).
5. Steps 1 and 2 above should have placed gmp library and header files in the folders '/usr/local/lib/' and '/usr/local/include/'; if they were placed elsewhere you will need to alter the file 'pyminksum-0.11/setup.py': change the lines `gmp_include = '/usr/local/include'` and `gmp_lib = '/usr/local/lib'` to point to the appropriate include and library directories.
6. In the terminal, navigate to the folder 'pyminksum-0.11/' and execute the command `python setup.py install`.
7. You should now be able to use `import setbasedmethods` in Python to import the package.

### Demonstrations

We'll first graphically illustrate some zonotopes (computed with PyMINKSUM) to depict what they are outside of the mathematical definition presented above.

In [ ]:
%matplotlib inline
#ms = MinkSum
import Polytopes as ms
import zonotopes as zn
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
import time

We start with the most easily visualized example: a zonotope where $m=3$ and $n=2$. This is a projection of a rotated cube into a plane. To get some idea of what's happening, imagine taking a cube sitting on a table, rotating it in all three dimensions, and looking at it from above; the outline of the shape you see is the zonotope. We would thus expect to see six (out of eight) of the cube's edges on the exterior of the cube, and two on the interior.

The code below generates a random generator matrix, computes polytopes defining the line segments whose Minkowski sum is the zonotope, computes the zonotope with the Minkowski sum method, and plots its vertices (large blue dots) along with all the cube's projected vertices (small red dots).

In [ ]:
#Random generator matrix
gens = np.random.uniform(-1, 1, (2, 3))

#This is a matrix whose columns are all 3-combinations of -1 and 1
X = np.array([[-1,-1,-1,-1,1,1,1,1],[-1,-1,1,1,-1,-1,1,1],[-1,1,-1,1,-1,1,-1,1]])

#These are all the projected vertices of the polytope
verts = gens.dot(X)

#Polytopes representing the line segments
poly_list = []
for i in range(3):
    poly_list.append(ms.Polytope(np.hstack((-1*gens[:,i][:,None], 1*gens[:,i][:,None])).T))

#These are the new zonotope vertices
NV = ms.MinkSum(poly_list)


#Get vertices adjacent to each vertex for plotting
adj_verts = []
for i in range(X.shape[1]):
    adj_verts.append([])
    a = np.copy(X[:,i])
    adj_verts[i].append(gens.dot(a.reshape((3, 1))))
    for j in range(X.shape[0]):
        a = np.copy(X[:,i])
        a[j] *= -1
        adj_verts[i].append(gens.dot(a.reshape((3, 1))))

In [ ]:
#This plots all vertices (small red circles) and the vertices of the zonotope (large blue 
#circles)

plt.figure(figsize=(7, 7))
plt.plot(NV[:,0], NV[:,1], 'bo', ms=20)
plt.plot(verts.T[:,0], verts.T[:,1], 'ro', ms=10)
plt.xlim([np.amin(NV[:,0]) - .3, np.amax(NV[:,0]) + .3])
plt.ylim([np.amin(NV[:,1]) - .3, np.amax(NV[:,1]) + .3])
plt.grid(True)

#Plot lines between adjacent vertices for illustration
for v in verts.T:
    if list(v) not in [list(NV[k]) for k in range(NV.shape[0])]:
        back_vertex = v
        break
for vert_set in adj_verts:
    for i in range(len(vert_set)):
        x1 = vert_set[0][0]; x2 = vert_set[i][0]
        y1 = vert_set[0][1]; y2 = vert_set[i][1]
        if list(back_vertex) in [[x1, y1], [x2, y2]]:
            plt.plot([x1, x2], [y1, y2], 'r:', linewidth=1)
        else:
            plt.plot([x1, x2], [y1, y2], 'r-', linewidth=1)

We now repeat as above, but with $m=5$ and $m=10$ for demonstration.

In [ ]:
#m is the larger dimension; e.g. if m=5, we project a rotated 5-D cube onto R^2

m = 5
gens = np.random.uniform(-1, 1, (2, m))
iterates = product([-1, 1], repeat=m)
X = np.empty((m, 1))
for row in iterates: X = np.hstack((X, np.array(row).reshape(m, 1)))
X = X[:,1:]
verts = gens.dot(X)
poly_list = []
for i in range(m):
    poly_list.append(ms.Polytope(np.hstack((-1*gens[:,i][:,None], 1*gens[:,i][:,None])).T))
NV = ms.MinkSum(poly_list)

In [ ]:
#This plots all vertices (small red circles) and the vertices of the zonotope (large blue 
#circles)

plt.figure(figsize=(7, 7))
plt.plot(NV[:,0], NV[:,1], 'bo', ms=20)
plt.plot(verts.T[:,0], verts.T[:,1], 'ro', ms=10)
plt.xlim([np.amin(NV[:,0]) - 1, np.amax(NV[:,0]) + 1])
plt.ylim([np.amin(NV[:,1]) - 1, np.amax(NV[:,1]) + 1])
plt.grid(True)

In [ ]:
m = 10
gens = np.random.uniform(-1, 1, (2, m))
iterates = product([-1, 1], repeat=m)
X = np.empty((m, 1))
for row in iterates: X = np.hstack((X, np.array(row).reshape(m, 1)))
X = X[:,1:]
verts = gens.dot(X)
poly_list = []
for i in range(m):
    poly_list.append(ms.Polytope(np.hstack((-1*gens[:,i][:,None], 1*gens[:,i][:,None])).T))
NV = ms.MinkSum(tuple(poly_list))

In [ ]:
plt.figure(figsize=(7, 7))
plt.plot(NV[:,0], NV[:,1], 'bo', ms=20)
plt.plot(verts.T[:,0], verts.T[:,1], 'ro', ms=10)
plt.xlim([np.amin(NV[:,0]) - 1, np.amax(NV[:,0]) + 1])
plt.ylim([np.amin(NV[:,1]) - 1, np.amax(NV[:,1]) + 1])
plt.grid(True)

In [ ]:
#Projecting a 20-D cube onto R^3 and timing for demonstration of efficacy
b = time.time()


m = 20
n = 3

#Random generator matrix
gens = np.random.uniform(-1, 1, (n, m))

poly_list = []

for i in range(m):
    poly_list.append(ms.Polytope(np.hstack((-1*gens[:,i][:,None], 1*gens[:,i][:,None])).T))

NV = ms.MinkSum(poly_list)

e = time.time()
print "took {} seconds or {} minutes".format(e-b, (e-b)/60.)

In [ ]:
b = time.time()

m = 5
n_list = np.array([2, 3, 4, 5])
max_ord = 5
p = np.arange(1, max_ord+.5, .5)
P = np.array(10**p, dtype=int)
n_sim = 5
true_zonotopes = []
errors = np.zeros((len(p), len(n_list), n_sim))

for n in n_list:
    gens = np.random.uniform(-1, 1, (n, m))
    poly_list = []
    for i in range(m):
        poly_list.append(ms.Polytope(np.hstack((-1*gens[:,i][:,None], 
                                                 1*gens[:,i][:,None])).T))
    Z = ms.MinkSum(poly_list)
    true_zonotopes.append(Z)
    for j in range(n_sim):
        V = []
        k = 0
        ord_V = zn.nzv(m, n)
        for i in range(P[-1]):
            x = np.random.multivariate_normal(np.zeros(n), np.eye(n))
            v_plus = gens.dot(np.sign(gens.T.dot(x)))
            if list(v_plus) not in V:
                V.append(list(v_plus)); V.append(list(-v_plus)); k += 2
            if i+1 in P:
                try:
                    errors[int(np.argwhere(P==i+1)), int(np.argwhere(n_list==n)), j] =\
                        zn.zonotope_hausdist(Z, np.array(V))
                except: pass
            if k == ord_V: break
                
e = time.time()
print "took {} seconds or {} minutes".format(e-b, (e-b)/60)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 14)); axes = axes.reshape(4).squeeze()

for i in range(len(n_list)):
    for n in range(n_sim):
        axes[i].loglog(P, errors[:,i,n], 'ko-')
        axes[i].grid(True)
        axes[i].set_xlim(P[0], P[-1]); axes[i].set_ylim(10**-4, 10**1)
        axes[i].set_xlabel(r"$n = {}$".format(n_list[i]), fontsize=16) 
        axes[i].set_ylabel("Error", fontsize=16)

In [ ]:
b = time.time()

m = 10
n_list = np.array([2, 3, 4, 5])
max_ord = 5
p = np.arange(1, max_ord+.5, .5)
P = np.array(10**p, dtype=int)
n_sim = 5
true_zonotopes = []
errors = np.zeros((len(p), len(n_list), n_sim))

for n in n_list:
    gens = np.random.uniform(-1, 1, (n, m))
    poly_list = []
    for i in range(m):
        poly_list.append(ms.Polytope(np.hstack((-1*gens[:,i][:,None], 
                                                 1*gens[:,i][:,None])).T))
    Z = ms.MinkSum(poly_list)
    true_zonotopes.append(Z)
    for j in range(n_sim):
        V = []
        k = 0
        ord_V = zn.nzv(m, n)
        for i in range(P[-1]):
            x = np.random.multivariate_normal(np.zeros(n), np.eye(n))
            v_plus = gens.dot(np.sign(gens.T.dot(x)))
            if list(v_plus) not in V:
                V.append(list(v_plus)); V.append(list(-v_plus)); k += 2
            if i+1 in P:
                try:
                    errors[int(np.argwhere(P==i+1)), int(np.argwhere(n_list==n)), j] =\
                        zn.zonotope_hausdist(Z, np.array(V))
                except: pass
            if k == ord_V: break
                
e = time.time()
print "took {} seconds or {} minutes".format(e-b, (e-b)/60)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 14)); axes = axes.reshape(4).squeeze()

for i in range(len(n_list)):
    for n in range(n_sim):
        axes[i].loglog(P, errors[:,i,n], 'ko-')
        axes[i].grid(True)
        axes[i].set_xlim(P[0], P[-1]); axes[i].set_ylim(10**-4, 10**1)
        axes[i].set_xlabel(r"$n = {}$".format(n_list[i]), fontsize=16) 
        axes[i].set_ylabel("Error", fontsize=16)

In [ ]:
b = time.time()

m_list = [5, 10]
n_list = [2, 3]

n_sim = 10**2
samps_needed = np.zeros((n_sim, len(m_list), len(n_list)))


for j in range(len(m_list)):
    for l in range(len(n_list)):
        m = m_list[j]; n = n_list[l]
        gens = np.random.uniform(-1, 1, (n, m))
        ord_V = zn.nzv(m, n)
        for i in range(n_sim):            
            V = []
            k = 0            
            n_samps = 0
            while k != ord_V:
                n_samps += 1
                x = np.random.multivariate_normal(np.zeros(n), np.eye(n))
                v_plus = gens.dot(np.sign(gens.T.dot(x)))
                if list(v_plus) not in V:
                    V.append(list(v_plus)); V.append(list(-v_plus)); k += 2
            samps_needed[i,j,l] = n_samps
                
e = time.time()
print "took {} seconds or {} minutes".format(e-b, (e-b)/60)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 14)); axes = axes.reshape(4).squeeze()
m_n_list = [[0, 0], [0, 1], [1, 0], [1, 1]]

for i in range(len(m_n_list)):
    ij = m_n_list[i]
    m = m_list[ij[0]]; n = n_list[ij[1]]
    axes[i].hist(np.log10(samps_needed[:,ij[0],ij[1]]), bins=15)
    axes[i].grid(True)
    axes[i].set_xlim(0, 9); axes[i].set_ylim(0, n_sim/4)
    axes[i].set_xlabel(r"$m = {}, n = {}$".format(m, n), fontsize=16) 
    axes[i].set_ylabel("Samples Needed", fontsize=16)